In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("data/incidents.csv")
df.head()


,incident_id,title,description,service_name,severity,time_to_resolve_minutes,is_sla_breached,root_cause_category
0,1,Issue 1 in system,Resource related problem detected,Inventory,P3,71,1,Database
1,2,Issue 2 in system,Resource related problem detected,Search,P3,107,1,Resource
2,3,Issue 3 in system,Config related problem detected,Inventory,P2,113,1,Database
3,4,Issue 4 in system,Deployment related problem detected,Payments,P1,97,0,Config
4,5,Issue 5 in system,Config related problem detected,Inventory,P4,157,1,Deployment


In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   incident_id              100 non-null    int64 
 1   title                    100 non-null    object
 2   description              100 non-null    object
 3   service_name             100 non-null    object
 4   severity                 100 non-null    object
 5   time_to_resolve_minutes  100 non-null    int64 
 6   is_sla_breached          100 non-null    int64 
 7   root_cause_category      100 non-null    object
dtypes: int64(3), object(5)
memory usage: 6.4+ KB


In [4]:
df.describe(include="all")


,incident_id,title,description,service_name,severity,time_to_resolve_minutes,is_sla_breached,root_cause_category
count,100.000000,100,100,100,100,100.000000,100.00,100
unique,NaN,100,5,5,4,NaN,NaN,5
top,NaN,Issue 1 in system,Resource related problem detected,Inventory,P4,NaN,NaN,Database
freq,NaN,1,27,29,34,NaN,NaN,28
mean,50.500000,NaN,NaN,NaN,NaN,105.500000,0.55,NaN
std,29.011492,NaN,NaN,NaN,NaN,48.683579,0.50,NaN
min,1.000000,NaN,NaN,NaN,NaN,16.000000,0.00,NaN
25%,25.750000,NaN,NaN,NaN,NaN,60.000000,0.00,NaN
50%,50.500000,NaN,NaN,NaN,NaN,113.000000,1.00,NaN
75%,75.250000,NaN,NaN,NaN,NaN,147.250000,1.00,NaN


Feature Engineering

In [5]:
severity_map = {"P1": 4, "P2": 3, "P3": 2, "P4": 1}
df["severity_encoded"] = df["severity"].map(severity_map)

df["is_sla_breached"] = df["is_sla_breached"].astype(int)

df["resolution_time_norm"] = df["time_to_resolve_minutes"] / df["time_to_resolve_minutes"].max()

df.head()


,incident_id,title,description,service_name,severity,time_to_resolve_minutes,is_sla_breached,root_cause_category,severity_encoded,resolution_time_norm
0,1,Issue 1 in system,Resource related problem detected,Inventory,P3,71,1,Database,2,0.394444
1,2,Issue 2 in system,Resource related problem detected,Search,P3,107,1,Resource,2,0.594444
2,3,Issue 3 in system,Config related problem detected,Inventory,P2,113,1,Database,3,0.627778
3,4,Issue 4 in system,Deployment related problem detected,Payments,P1,97,0,Config,4,0.538889
4,5,Issue 5 in system,Config related problem detected,Inventory,P4,157,1,Deployment,1,0.872222


In [6]:
#Define Target
df["risk_label"] = np.where(
    (df["severity_encoded"] >= 3) | (df["is_sla_breached"] == 1),
    1,  # High Risk
    0   # Low Risk
)


In [7]:
#Check Class Imbalance
df["risk_label"].value_counts(normalize=True)


risk_label
1    0.75
0    0.25
Name: proportion, dtype: float64

In [8]:
X = df[["severity_encoded", "resolution_time_norm", "is_sla_breached"]]
y = df["risk_label"]


Train First Model (Logistic)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [10]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00        15

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



GRADIENT BOOSTING FOR INCIDENT RISK
The goal is to-

Use Gradient Boosting (XGBoost/LightGBM) to predict high-risk incidents.
We are building a model that answers:

“Given incident details, is this a High Risk incident?”

High risk =

1.Severity high OR

2.SLA breached OR

3.Very slow resolution

This aligns with:

“Forecast the probability of high-priority incidents for proactive alerting”


In [11]:
#Prepare the Risk Label (Target)
import numpy as np

df["risk_label"] = np.where(
    (df["severity_encoded"] >= 3) | (df["is_sla_breached"] == 1),
    1,  # High Risk
    0   # Low Risk
)

df["risk_label"].value_counts(normalize=True)


risk_label
1    0.75
0    0.25
Name: proportion, dtype: float64

In [12]:
#Select Features
features = [
    "severity_encoded",
    "resolution_time_norm",
    "is_sla_breached"
]

X = df[features]
y = df["risk_label"]


In [13]:
#Train-Test Split (Stratified)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

### Why Gradient Boosting?

Gradient Boosting builds many small decision trees, 
where each new tree corrects the mistakes of the previous ones.

It works extremely well on:
- Structured tabular data
- Feature-engineered datasets
- Non-linear relationships
- Enterprise datasets with mixed signals


In [14]:
#Train Gradient Boosting Model (XGBoost style using sklearn)
from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

gb_model.fit(X_train, y_train)


GradientBoostingClassifier(random_state=42)

You built 100 trees

Each tree is small

Each tree fixes previous errors

Together they become a strong predictor

In [15]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = gb_model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[ 5  0]
 [ 0 15]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00        15

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [16]:
#Feature Importance (Explainability)-->This answers:“Why did the model decide this?

import pandas as pd

feature_importance = pd.Series(
    gb_model.feature_importances_,
    index=features
).sort_values(ascending=False)

feature_importance


severity_encoded        5.490196e-01
is_sla_breached         4.509804e-01
resolution_time_norm    7.405528e-17
dtype: float64

### Why XGBoost / LightGBM?

These are optimized implementations of Gradient Boosting that:
- Handle missing values automatically
- Are much faster
- Prevent overfitting using regularization
- Scale to large datasets
- Are widely used in industry


In [17]:
!pip install xgboost lightgbm


In [18]:
from xgboost import XGBClassifier


In [50]:
from sklearn.model_selection import train_test_split

features = [
    "severity_encoded",
    "resolution_time_norm",
    "is_sla_breached"
]

X = df[features]
y = df["risk_label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)



In [20]:
y_train.value_counts()


risk_label
1    60
0    20
Name: count, dtype: int64

In [21]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    n_estimators=150,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight= (y_train.value_counts()[0] / y_train.value_counts()[1]),
    random_state=42,
    eval_metric="logloss"
)

xgb_model.fit(X_train, y_train)


C:\Users\shraw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The XGBClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=150,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [22]:
xgb_model = XGBClassifier(
    n_estimators=150,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight= (y_train.value_counts()[0] / y_train.value_counts()[1]),
    random_state=42,
    eval_metric="logloss"
)


In [53]:
y_pred_xgb = xgb_model.predict(X_test)

print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))


NotFittedError: need to call fit or load_model beforehand

In [52]:
pd.Series(
    xgb_model.feature_importances_,
    index=features
).sort_values(ascending=False)


NotFittedError: need to call fit or load_model beforehand

In [26]:
#LIGHTGBM MODEL

from lightgbm import LGBMClassifier


In [27]:
lgbm_model = LGBMClassifier(
    n_estimators=150,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight="balanced",
    random_state=42
)


In [28]:
lgbm_model.fit(X_train, y_train)

y_pred_lgbm = lgbm_model.predict(X_test)

print(confusion_matrix(y_test, y_pred_lgbm))
print(classification_report(y_test, y_pred_lgbm))


[LightGBM] [Info] Number of positive: 60, number of negative: 20
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000445 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 32
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Light

In [29]:
pd.Series(
    lgbm_model.feature_importances_,
    index=features
).sort_values(ascending=False)


resolution_time_norm    183
severity_encoded         65
is_sla_breached          49
dtype: int32

In [30]:
print("XGBoost:")
print(confusion_matrix(y_test, y_pred_xgb))

print("\nLightGBM:")
print(confusion_matrix(y_test, y_pred_lgbm))


XGBoost:


NameError: name 'y_pred_xgb' is not defined

In [31]:
from sklearn.metrics import classification_report

print("XGBoost Report:")
print(classification_report(y_test, y_pred_xgb, output_dict=True)["1"])

print("\nLightGBM Report:")
print(classification_report(y_test, y_pred_lgbm, output_dict=True)["1"])


XGBoost Report:


NameError: name 'y_pred_xgb' is not defined

## Model Comparison: XGBoost vs LightGBM

Both XGBoost and LightGBM produced similar performance on the current dataset.

### Observations:
- Recall for the High Risk class was the same for both models
- Confusion matrices showed identical False Negatives (missed critical incidents)
- Feature importance rankings were also similar

### Conclusion:
Since both models perform equally well, this indicates that the **engineered features are strong and highly predictive**.  
In such cases, model choice becomes secondary to feature quality.

In real-world scenarios with noisier and larger datasets, performance differences between XGBoost and LightGBM are expected to emerge.


ROOT CAUSE ANALYSIS (RCA) USING NLP + TF-IDF

This part directly matches the proposal:

“Automatically predict the root cause of incidents based on title, description, and metadata.”

In [32]:
#Prepare Text Data
df["text"] = df["title"] + " " + df["description"]
df[["text", "root_cause_category"]].head()


,text,root_cause_category
0,Issue 1 in system Resource related problem det...,Database
1,Issue 2 in system Resource related problem det...,Resource
2,Issue 3 in system Config related problem detected,Database
3,Issue 4 in system Deployment related problem d...,Config
4,Issue 5 in system Config related problem detected,Deployment


In [33]:
#Convert Text → Numbers (TF-IDF)

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    stop_words="english",
    max_features=500
)

X_text = tfidf.fit_transform(df["text"])
y_rca = df["root_cause_category"]


Human language → numerical vectors

TF-IDF gave:

Higher weight to rare but important words

Lower weight to common words like the, is, in

This is why TF-IDF is perfect for incident RCA.

In [34]:
from sklearn.model_selection import train_test_split

X_train_text, X_test_text, y_train_rca, y_test_rca = train_test_split(
    X_text, y_rca,
    test_size=0.2,
    stratify=y_rca,
    random_state=42
)


In [35]:
#Multi-Class Classifier

#We use Logistic Regression for:

#Interpretability

#Stability

#Multi-class support

from sklearn.linear_model import LogisticRegression

rca_model = LogisticRegression(max_iter=1000)
rca_model.fit(X_train_text, y_train_rca)


LogisticRegression(max_iter=1000)

In [36]:
from sklearn.metrics import classification_report

y_pred_rca = rca_model.predict(X_test_text)
print(classification_report(y_test_rca, y_pred_rca))

#Focus on:F1-score per root cause, Which class performs best/worst





              precision    recall  f1-score   support

      Config       0.00      0.00      0.00         4
    Database       0.19      0.60      0.29         5
  Deployment       0.00      0.00      0.00         4
     Network       0.00      0.00      0.00         3
    Resource       0.00      0.00      0.00         4

    accuracy                           0.15        20
   macro avg       0.04      0.12      0.06        20
weighted avg       0.05      0.15      0.07        20



C:\Users\shraw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shraw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shraw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati

THIS IS MULTI-CLASS CLASSIFICATION

Unlike risk prediction (binary):
Here the model chooses between:

Database

Network

Deployment

Resource

Config

That’s multi-class ML in action.

In [37]:
#Most Important Keywords per Root Cause
import numpy as np

feature_names = tfidf.get_feature_names_out()

for i, class_label in enumerate(rca_model.classes_):
    top_words = np.argsort(rca_model.coef_[i])[-5:]
    print(f"\nTop words for {class_label}:")
    print([feature_names[j] for j in top_words])



Top words for Config:
['67', '91', '46', '58', 'deployment']

Top words for Database:
['32', '54', '56', '33', '93']

Top words for Deployment:
['38', '69', '45', '92', '77']

Top words for Network:
['28', '83', '49', '68', '99']

Top words for Resource:
['41', '81', '61', '25', '86']


## Root Cause Analysis using NLP

This module predicts the root cause of incidents based on textual data 
(title and description) using TF-IDF and multi-class classification.

Key advantages:
- Converts unstructured text into structured intelligence
- Enables automatic root cause prediction
- Provides explainability using keyword importance


###SIMILAR INCIDENT SEARCH (VECTOR SIMILARITY)

In [38]:
#Create Vector Representations (Using TF-IDF)
incident_vectors = tfidf.transform(df["text"])


In [39]:
#Build Similarity Search (Cosine Similarity)
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(incident_vectors)


In [40]:
def find_similar_incidents(index, top_n=5):
    sim_scores = list(enumerate(similarity_matrix[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Remove the incident itself (first one)
    sim_scores = sim_scores[1:top_n+1]
    
    return sim_scores


In [41]:
similar = find_similar_incidents(0)

for idx, score in similar:
    print(f"Incident {idx} | Similarity: {score:.2f}")
    print(df.iloc[idx]["text"])
    print("Root cause:", df.iloc[idx]["root_cause_category"])
    print("-"*50)


Incident 1 | Similarity: 1.00
Issue 2 in system Resource related problem detected
Root cause: Resource
--------------------------------------------------
Incident 13 | Similarity: 0.52
Issue 14 in system Resource related problem detected
Root cause: Database
--------------------------------------------------
Incident 19 | Similarity: 0.52
Issue 20 in system Resource related problem detected
Root cause: Database
--------------------------------------------------
Incident 22 | Similarity: 0.52
Issue 23 in system Resource related problem detected
Root cause: Database
--------------------------------------------------
Incident 23 | Similarity: 0.52
Issue 24 in system Resource related problem detected
Root cause: Config
--------------------------------------------------


## Similar Incident Retrieval using Vector Similarity

This module retrieves semantically similar historical incidents 
based on TF-IDF vector similarity.

It enables:
- Knowledge reuse from past incidents
- Faster troubleshooting
- Foundation for RAG-based systems in operations


In [42]:
from collections import Counter

def rag_style_root_cause(new_text, top_n=5):
    # Convert new incident text to vector
    new_vector = tfidf.transform([new_text])
    
    # Compute similarity with all past incidents
    similarities = cosine_similarity(new_vector, incident_vectors)[0]
    
    # Get top similar incidents
    top_indices = similarities.argsort()[-top_n:][::-1]
    
    # Collect their root causes
    top_root_causes = df.iloc[top_indices]["root_cause_category"]
    
    # Most common root cause among them
    predicted_cause = Counter(top_root_causes).most_common(1)[0][0]
    
    return predicted_cause, df.iloc[top_indices][["text", "root_cause_category"]]


In [43]:
test_incident = "Payment service database connection timeout"

predicted, evidence = rag_style_root_cause(test_incident)

print("Predicted Root Cause:", predicted)
print("\nEvidence from similar incidents:")
evidence


Predicted Root Cause: Database

Evidence from similar incidents:


,text,root_cause_category
7,Issue 8 in system Database related problem det...,Database
6,Issue 7 in system Database related problem det...,Resource
97,Issue 98 in system Database related problem de...,Database
82,Issue 83 in system Database related problem de...,Network
73,Issue 74 in system Database related problem de...,Network


## Vector Database using FAISS and Persistence using PKL

To enable fast and scalable similarity search over incident vectors, 
we use FAISS (Facebook AI Similarity Search) as a vector search engine.

### Why FAISS?
FAISS is designed for efficient similarity search over high-dimensional vectors.
It is used when:
- The dataset grows to thousands or millions of incidents
- Brute-force similarity computation becomes slow
- Real-time retrieval is required

In this project, FAISS is used to:
- Store TF-IDF embeddings of incident text
- Retrieve the most similar historical incidents for a new query
- Enable RAG-style reasoning for root cause recommendation

### Persistence using PKL (Pickle)

To avoid rebuilding the vector index every time the system runs,
we serialize (save) the FAISS index using PKL.

This allows:
- Faster system startup
- Reuse of trained vector indexes
- Deployment-ready vector storage

### Enterprise Relevance

In production systems:
- FAISS/Chroma act as vector databases
- Pickle/Blob storage is used for persistence
- This forms the foundation for scalable RAG-based AI systems in IT Operations


In [44]:
!pip install faiss-cpu


In [45]:
import faiss
import numpy as np

# Convert sparse TF-IDF to dense
dense_vectors = incident_vectors.toarray().astype("float32")

# Create FAISS index
index = faiss.IndexFlatL2(dense_vectors.shape[1])
index.add(dense_vectors)

print("Total vectors in index:", index.ntotal)


Total vectors in index: 100


In [46]:
def faiss_search(new_text, top_k=5):
    new_vec = tfidf.transform([new_text]).toarray().astype("float32")
    distances, indices = index.search(new_vec, top_k)
    
    return df.iloc[indices[0]][["text", "root_cause_category"]]


In [47]:
faiss_search("High CPU in payment service after deployment")


,text,root_cause_category
3,Issue 4 in system Deployment related problem d...,Config
5,Issue 6 in system Deployment related problem d...,Config
10,Issue 11 in system Deployment related problem ...,Deployment
15,Issue 16 in system Deployment related problem ...,Config
17,Issue 18 in system Deployment related problem ...,Config


In [48]:
import pickle

with open("faiss_index.pkl", "wb") as f:
    pickle.dump(index, f)


In [49]:
with open("faiss_index.pkl", "rb") as f:
    index = pickle.load(f)
